# Repeated Measures

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QAHRoddur/JWAS/blob/main/Examples/repeated_measures.ipynb)

This notebook is auto-generated from the JWAS.jl wiki page.


In [ ]:
using Pkg
Pkg.add("JWAS")
Pkg.precompile()
using JWAS


# Repeated Measures (single-trait and multi-trait)
  * [Repeated Measures (Genomic data)](#st_g)
  * [Repeated Measures](#st)

## Repeated Measures (Genomic data)
<div id="st_g" />


### Data Description

Below we simulate a dataset with repeated measures to demonstrate the data format for repeatability models in JWAS. We duplicate the dataset and add noise to create multiple records per individual. A column "pe" (same as "ID") is added for permanent environmental effects.


In [ ]:
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets,StatsBase
phenofile  = "../data/phenotypes.txt"
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstring=["NA"])
nobs = size(phenotypes,1)
phenotypes = sort([phenotypes; phenotypes; phenotypes[sample(1:nobs, min(nobs,3), replace=false),:]])
phenotypes[!,:y1]=phenotypes[!,:y1]+randn(size(phenotypes,1))
phenotypes[!,:y2]=phenotypes[!,:y2]+randn(size(phenotypes,1))
phenotypes[!,:y3]=phenotypes[!,:y3]+randn(size(phenotypes,1))
phenotypes[!,:pe]=phenotypes[!,:ID]
show(phenotypes, allcols=true)


### JWAS Analysis


In [ ]:
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets,StatsBase

# Step 2: Read data 
pedfile    = "../data/pedigree.txt"
genofile   = "../data/genotypes.txt"
pedigree   = get_pedigree(pedfile,separator=",",header=true);
genotypes  = get_genotypes(genofile,separator=',',method="BayesC");
first(phenotypes,5)


In [ ]:
# Step 3: Build Model Equations
model_equation  ="y1 = intercept + x1 + ID + pe + genotypes"
model = build_model(model_equation);

# Step 4: Set Factors or Covariates
set_covariate(model,"x1");

# Step 5: Set Random or Fixed Effects
set_random(model,"ID",pedigree); #polygenic effects
set_random(model,"pe"); #i.i.d. permanent environmental effects to account for the covariance between observations from the same individual

# Step 6: Run Analysis
out=runMCMC(model,phenotypes);

# Step 7: Check Results
out["EBV_y1"]  # estimated breeding values


## Repeated Measures
<div id="st_g" />

To run analysis without genomic data, just remove "genotypes" in the model_equation from the script above.
